In [1]:
import pandas as pd
import re
import textwrap
from common.evaluate_helpers import preprocess_text as preprocess_text_bm25
from nltk.tokenize import word_tokenize
from tabulate import tabulate

# Set display options to show full content of each column
pd.set_option("display.max_colwidth", None)


def preprocess_text(text: str) -> str:
    if type(text) != str:
        return ""

    # Remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    # Remove unnecessary whitespaces
    text = re.sub(r"\s+", " ", text).strip()

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stop words and stem
    return " ".join([word for word in tokens])

In [2]:
MINISTRY = "minbuza"

woo_data = pd.read_csv(f"./docs_ministries_full/{MINISTRY}/woo_merged.csv.gz")

### Display all corresponding documents

In [3]:
DOSSIER = "nl.mnre1013.2i.2020.1"
TYPES = ["all"]
# TYPES = ["verzoek", "besluit"]
# TYPES = ["bijlage", "inventaris"]
RELEVANT_COLUMNS = ["dossier_id", "type", "source"]
RELEVANT_DISPLAY_COLUMNS = ["dossier_id", "document_id", "type", "source"]


if "all" in TYPES:
    filtered_data = woo_data[(woo_data["dossier_id"] == DOSSIER)]
else:
    filtered_data = woo_data[(woo_data["type"].isin(TYPES)) & (woo_data["dossier_id"] == DOSSIER)]

# Explicitly copy the filtered DataFrame to avoid SettingWithCopyWarning when modifying it.
filtered_data = filtered_data[RELEVANT_DISPLAY_COLUMNS].copy()
filtered_data.drop_duplicates(inplace=True, subset=RELEVANT_COLUMNS)
filtered_data.reset_index(inplace=True, drop=True)

print(tabulate(filtered_data, headers=RELEVANT_DISPLAY_COLUMNS, tablefmt="psql"))

+----+-----------------------+------------------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | dossier_id            | document_id                  | type    | source                                                                                                                                                             |
|----+-----------------------+------------------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | nl.mnre1013.2i.2020.1 | nl.mnre1013.2i.2020.1.misc.1 | besluit | https://open.overheid.nl/repository/ronl-caed7da5-d674-493b-a9c4-339aa1877223/1/pdf/Besluit%20op%20Wob-verzoek%20Interlandelijke%20adopties%20Sri%20Lanka.pdf      |
|  1 | nl.mnre1013.2i.2020.1 | nl.mnre1013.2i.2020.1

### Display the bodytext of a page

In [4]:
PAGE = "nl.mnre1013.2i.2020.1.misc.4.pagina.29"
RELEVANT_DISPLAY_COLUMNS = ["bodyText"]


filtered_data = woo_data[(woo_data["page_id"] == PAGE)]
filtered_data = filtered_data[RELEVANT_DISPLAY_COLUMNS].copy()
filtered_data.reset_index(inplace=True, drop=True)

all_text = " ".join(filtered_data["bodyText"].astype(str))
# processed_text = preprocess_text(all_text)
processed_text = " ".join(preprocess_text_bm25(all_text))

print(textwrap.fill(processed_text, width=140))

den opgeborgen per teleloon gesproken dossier wekk codeb bevallen dienen verg le wor over inhoud codeb mag verwijz bencnt i tenzi bened
verdenor worn vormen y ee ingekomen jan agenda no dossier dop ki ontvangen codebericht datum van ontvangst referenti no dirafd
informatiekopi bestemd voor as jar al cave de chena on dal wpe wer onl warn aber afkomstig van cri en justiti zullen trachten deze kwesti
voor wat betreft de nederlands betrokkenheid meer te konkretiseren onder ander de wijz waarop de kinderen nederland binnenkomen is
onduidelijk zou het op prij stellen van u te vernemen hoe de visumafgift duir cg amman verloopt met name of de uiterst kort termijn die ce
adoptiequd in jordani doorbrengen in óvereenstemn te oo grengen is met het vereist dat visumaanvragen eerst aan u dienen te worden
voorgelegd valt voort door u na te gaan wie werkzaam zijn op het srila e consulaat te amman en met name of genoemd aan dat consulaat
verbonden is houd mij overigen aanbevolen voor elk aanvullend inf

: 

### Display the bodytext of a document

In [78]:
DOCUMENT = "nl.mnre1058.2i.2018.37.doc.2"
RELEVANT_DISPLAY_COLUMNS = ["bodyText"]


filtered_data = woo_data[(woo_data["document_id"] == DOCUMENT)]
filtered_data = filtered_data[RELEVANT_DISPLAY_COLUMNS].copy()
filtered_data.reset_index(inplace=True, drop=True)

# Apply preprocessing to each text entry
filtered_data["bodyText"] = filtered_data["bodyText"].apply(preprocess_text)

all_text = " ".join(filtered_data["bodyText"])

print(textwrap.fill(all_text, width=140))

Dienst Justitiële Inrichtingen Ministerie van Veiligheid en Justitie Retouradres Postbus 30132 2500 GC Den Haag Directie Beleid
Bestuursondersteuning Juridische Zaken ADR Nieuwmedia Turfmarkt 147 2511 DP Den Haag Postbus 30132 2500 CC Den Haag www dj 1 fl1
Contactpersoon Alleen oer email Ons kenmerk 2164748 Datum 9 mei 2018 Bijlage Onderwerp Uw Wob verzoek om inzage in
medewerkerstevredenheidsonderzoeken Bq beantwoording de datum en ons kenmerk vermelden Wilt u slechts één zaak in uw brief behandelen
Geachte heel__________________ Bij brief van 29 november 2017 door mij ontvangen op 1 december 2017 verzoekt u op grond van de Wet
openbaarheid bestuur de Wob om inzage in alle gerapporteerde medewerkerstvredenheidsonderzoeken hierna MTOs die zijn gehouden onder de
medewerkers van de Pls Haaglanden en Pl Vught in de jaren 2016 en 2017 Bij brief van 8 december 2017 heb ik de ontvangst van uw verzoek
bevestigd en bij brief van 22 december 2017 heb ik het besluit op uw verzoek verdaagd met vi